Формирование pkl файлов исходных (data/external) и из БД для дальнейшего использования 


Делать однократно, если ранее не были созданы pkl файлы в папке interm/external_pkl

In [1]:
pre = '../../' # путь до главной директории проекта
%run ../init.ipynb #подгрузить нужные библиотеки
%run ../../params.ipynb #параметры как логин пароль к БИ, параметры расчёта ( в нашем случае их нет)

Исходные внешние данные:

dataset_local_application_inf_ssf.01.07.2022.pkl - должен быть заархивирован и сохранен в директории 'data/external/target_data.tar.gz'

Поскольку dataset_local_application_inf_ssf.\*\*.\*\*.\*\*\*\*.pkl имеет в названии отчетную дату, на которую формируют эту таблицу, то во избежание изменение кода, после разархивации, файл переименовывается в target_dataset.pkl, при это отчетная дата записывается в виде столбца 'date_of_report' таблицы для отладочных целей.

In [2]:
def read_target_ext():
    
    name = pre + 'data/external/target_data.tar.gz'
    extract_all(name, pre +'data/interm/external_pkl')

    name = pre + "data/interm/external_pkl/dataset_local_application_inf_ssf.01.07.2022.pkl"
    target_data = pd.read_pickle(name)

    print("Прочитанный размер файла из target-class {}".format(target_data.shape))
    
    report_date_string = name[-14:-4] # последняя дата в выборке кредитной выборке, во фрод +12мес.
    date_of_report = datetime.strptime(report_date_string, '%d.%m.%Y') 
    target_data['date_of_report'] = date_of_report
    
    try:
        if os.path.isfile(name):
            os.remove(name)
    except:
        pass

    return target_data

target_data = read_target_ext()
target_data.to_pickle(pre+"data/interm/external_pkl/target_dataset.pkl", compression={'method':'zip'})
# target_data.head(1)

Прочитанный размер файла из target-class (252134, 94)


In [3]:
def MDL_APP():
    sql_query = """SELECT PL,  
                        DATE_ACTUAL, 
                        NL_NEW_PRODUCTNAME, 
                        O_NEW_TOTALFINANCINGSUM 
                    FROM ANALYST.MDL_APP_DAKSENOVA_010723
                """

    mdl_app = read_from_oracle(sql_query, title='mpl_app')

    return mdl_app


mdl_app = MDL_APP()
mdl_app.to_pickle(pre+"data/interm/external_pkl/mdl_app.pkl",
                  compression={'method': 'zip'})
# mdl_app.head(1)


Размер считанных данных mpl_app - (250698, 4)


In [4]:
def portfolio_read():
    sql_query = """SELECT LP.CONTRACT_KEY, LP.CONTRACT_APP_KEY, LP.CONTRACT_NUM, LP.LEASE_DATE, LP.REHIRING_DT, LP.APP_1C_NUM,
                        CLIENTS.INN, CLIENTS.SHORT_CLIENT_RU_NAM,  CONTRACTS.OPEN_DT, CONTRACTS.CLOSE_DT 
                    FROM
                    (SELECT AL.APP_1C_NUM,
                        AL.CONTRACT_NUM,
                        AL.REHIRING_DT,
                        AL.CONTRACT_KEY,
                        AL.CONTRACT_APP_KEY,
                        AL.LEASE_DATE
                    FROM DM.AUTO_LEASING_PORTFOLIO AL
                    INNER JOIN
                        ( SELECT CONTRACT_KEY, CONTRACT_APP_KEY, MAX(SNAPSHOT_DT) AS SNAPSHOT_DT
                        FROM DM.AUTO_LEASING_PORTFOLIO
                        WHERE AUTO_LEASING_PORTFOLIO_FLG = 1
                        AND REHIRING_DT <= &CUTOFF
                        AND LEASE_DATE <= &CUTOFF
                        GROUP BY CONTRACT_KEY, CONTRACT_APP_KEY
                        ) LST
                    ON AL.CONTRACT_KEY      = LST.CONTRACT_KEY
                    AND AL.CONTRACT_APP_KEY = LST.CONTRACT_APP_KEY
                    AND AL.SNAPSHOT_DT      = LST.SNAPSHOT_DT
                    ) LP
                    INNER JOIN
                    (SELECT CONTRACT_KEY, CLIENT_KEY, OPEN_DT, CLOSE_DT
                    FROM DWH.CONTRACTS
                    WHERE VALID_TO_DTTM = TO_DATE('01.01.2400','DD.MM.YYYY')
                    ) CONTRACTS
                    ON CONTRACTS.CONTRACT_KEY = LP.CONTRACT_KEY
                    LEFT JOIN
                    (SELECT DISTINCT CLIENT_KEY, INN, DWH.CLIENTS.SHORT_CLIENT_RU_NAM, CLIENT_CD
                    FROM DWH.CLIENTS
                    WHERE VALID_TO_DTTM = TO_DATE('01.01.2400','DD.MM.YYYY')
                    ) CLIENTS
                    ON CLIENTS.CLIENT_KEY = CONTRACTS.CLIENT_KEY
                """
                
    ДатаСреза = datetime(2023, 9, 14)
    
    df = read_from_oracle(sql_query, title='portfolio', params={'CUTOFF':ДатаСреза})
    
    df['REPORT_DT'] = ДатаСреза
    
    for col in ['CONTRACT_APP_KEY', 'APP_1C_NUM', 'CONTRACT_KEY']:
        df[col] = df[col].astype(str)
    
    for col in ['LEASE_DATE', 'CLOSE_DT', 'REHIRING_DT']:
        df[col] = pd.to_datetime(df[col], format='%Y.%m.%d', errors='coerce')
        
    return df

portfolio = portfolio_read()
portfolio.to_pickle(pre+"data/interm/external_pkl/portfolio.pkl", compression={'method': 'zip'})
# portfolio.head(1)


Размер считанных данных portfolio - (268031, 10)


In [5]:
def okved_read():
    sql_query = """ SELECT DISTINCT INN, 
                        CLIENT_KEY,
                        OKVED_CODE
                    FROM DWH.CLIENTS
                    WHERE VALID_TO_DTTM = TO_DATE('01.01.2400','DD.MM.YYYY')
                    AND OKVED_CODE IS NOT NULL
                    AND NVL(DWH.CLIENTS.DELETE_FLG, 0) !=1
                """
                
    df = read_from_oracle(sql_query, title='okved')
    
    return df

okved = okved_read()
okved.to_pickle(pre+"data/interm/external_pkl/okved.pkl", compression={'method': 'zip'})
# okved.head(1)

Размер считанных данных okved - (152687, 3)


In [6]:
def read_keys():
    sql_query =   """ 
                    SELECT CONTRACT_KEY,
                            CONTRACT_NUM,
                            CRM_CONTRACT_CD,
                            CODE1C_CD
                    FROM DWH.LEASING_CONTRACTS lc
                    WHERE VALID_TO_DTTM = TO_DATE('01.01.2400', 'dd.mm.yyyy')
                    """

    keys = read_from_oracle(sql_query, title='keys')
    # делаем заглавными
    keys['CRM_CONTRACT_CD']= [s.upper( ) if s is not None  else s for s in keys['CRM_CONTRACT_CD'] ]
    return keys

keys = read_keys()
keys.to_pickle(pre+"data/interm/external_pkl/keys.pkl", compression={'method': 'zip'})

# keys.head()

Размер считанных данных keys - (392265, 4)
